# 生成代码并自主运行计算

##  Python REPL

Python REPL（Read-Eval-Print Loop）是一个交互式编程环境，允许程序员立即执行代码并获取结果。REPL 这个术语来源于描述这种环境工作方式的四个步骤：

- Read（读取） - 读取用户输入的代码。
- Eval（求值） - 对输入的代码进行求值。
- Print（打印） - 打印求值的结果。
- Loop（循环） - 返回第一步，等待用户输入更多的代码。

在 Python 中，REPL 通常是通过命令行工具 python 或 python3（取决于安装的 Python 版本）访问的。启动这个工具后，用户可以直接输入 Python 代码，并立即看到代码执行的结果。



In [3]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL


python_repl = PythonREPL()

python_repl.run("print(1+1)")

Python REPL can execute arbitrary code. Use with caution.


'2\n'

In [4]:
from langchain_openai import ChatOpenAI
import os

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=4096)

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

promptFormat = """{query}

请根据上面的问题，生成python代码并计算出问题的答案，最后计算出的答案用print()打印出来，请直接返回python代码，并进行解释。
不要返回其他任何字符串。
"""

prompt = ChatPromptTemplate.from_template(promptFormat)

chain = prompt | llm | StrOutputParser()


In [7]:
query = "三箱苹果重45千克，一箱梨比一箱苹果重5千克，3箱梨重多少千克？"

for chunk in chain.stream({"query": query}):
    print(chunk, end="", flush=True)


apple_weight = 45 / 3 # 计算一箱苹果的重量
pear_weight = apple_weight + 5 # 计算一箱梨的重量
total_pear_weight = pear_weight * 3 # 计算三箱梨的总重量
print(total_pear_weight)

In [8]:
results = chain.invoke({"query": query})

def parsePython(codeStr: str):
    codeStr = results.replace("```python", "").replace("```", "")
    return codeStr


chain = prompt | llm | StrOutputParser() | parsePython 

chain.invoke({"query": query})

'apple_weight = 45 / 3 # 计算一箱苹果的重量\npear_weight = apple_weight + 5 # 计算一箱梨的重量\ntotal_pear_weight = pear_weight * 3 # 计算三箱梨的总重量\n\nprint(total_pear_weight) # 打印出答案'

In [9]:
python_repl.run(results)

'60.0\n'

In [10]:
chain = prompt | llm | StrOutputParser() | parsePython | python_repl.run

chain.invoke({"query": query})

'60.0\n'

In [11]:
for chunk in chain.stream({"query": query}):
    print(chunk, end="", flush=True)

60.0
